In [ ]:
import tensorflow as tf
import numpy as np
import math
import glob
import matplotlib
import os

import matplotlib.pyplot as plt
from matplotlib.pyplot import plot,savefig
from matplotlib.colors import ListedColormap

import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

import mylib as ml2
from mylib import LiftNet, create_LiftNet, create_Standard_LiftNet, Standard_LiftNet, create_Standard_LiftNet_CWRU
from dataset import *

from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import pandas as pd

import sklearn as sk
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.decomposition import PCA

import seaborn as sea

#setting offilne
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
#------------------------default parameters-----------------------#
test_rate = 0.2
epochs = 1000

lr=0.015
momentum=0.8

decay=0.01

validation_split=0.2
steps_per_epoch=1
validation_steps=1
bunch_steps = 100
snapshot = 500
bunch_steps = 100
snapshot = 500

result_save_path = '/media/silverbullet/data_and_programing_file/newpaper/result/transfer_learning_result'

print('set default parameters')

In [ ]:
#------------------------adjustable parameters-----------------------#
k_flod_num = [2,3,4,5,6,7,8,9,10]

circle_num = 2
steps = 1000
whether_use_CWRU_data = 0

whether_expansion_test_data = 1
whether_expansion_train_data = 1
expansion_data_number = 500
noise_scales = 0.01

LiftingNet_noise_scale = 0.01

whether_append_expansion_train_data = 1
whether_append_expansion_test_data = 1

if whether_use_CWRU_data==1:
    class_num = 6
else:
    class_num = 5
"""
if whether_use_CWRU_data==1:
    noise_scales = LiftingNet_noise_scale
"""
#artificial_feature_method: 1 is 19 features, 2 is 9 features
artificial_feature_method = 2

pca_parameters = 27

if whether_use_CWRU_data==1:
    result_files_last_name = 'CWRU_data_train_model_our_data_test.png'
    result_txt_last_name = 'CWRU_data_train_model_our_data_test.txt'
else:
    result_files_last_name = 'our_'+ str(circle_num)+'_circles_data_trains_model_CWRU_data_test.png'
    result_txt_last_name = 'our_'+ str(circle_num)+'_circles_data_trains_model_CWRU_data_test.txt'

label_name1 = ['NORMAL', 'IR', 'OR', 'BALL', 'JOINT']
label_name2 = ['NORMAL', 'BALL', 'IR', 'OR_3', 'OR_6', 'OR_12']

if whether_use_CWRU_data == 1:
    label_name = label_name2
else:
    label_name = label_name1


In [ ]:
#-------------------------set read name-----------------------------#

model_path = ['/media/silverbullet/data_and_programing_file/newpaper/code/code1226/model/circle_1_to_6_in_our_data/', 
              '/media/silverbullet/data_and_programing_file/newpaper/code/code1226/model/circle_7_to_16_in_our_data_with_0.01_noise_data/',
              '/media/silverbullet/data_and_programing_file/newpaper/code/code1226/model/model_in_CWRU_data/']
model_head_name = ['Standard_LiftingNet_',
                   'Standard_LiftingNet_use_expansion_data__with_',
                   'Standard_expansion_CWRU_data_LiftingNet__with_',
                  'Standard_LiftingNet_use_expansion_data_',
                  'Standard_expansion_CWRU_data_LiftingNet_']

data_path = ['/media/silverbullet/data_and_programing_file/newpaper/dataset/newdataset/','/media/silverbullet/data_and_programing_file/newpaper/dataset/CWRU/CWRUdataset']

if whether_use_CWRU_data == 1:
    cutsize = 1024
    input_shape = (cutsize,2)
    channel = 2
    dataset, label = load_CWRU_data(data_path[1])
    read_model_name = model_path[2]+model_head_name[2]+str(LiftingNet_noise_scale)+'_noise_'+str(cutsize)+'_data_the_'+str(steps)+'th_snapshot_with_'+str(steps_per_epoch)+'_steps_per_epoch.h5'
    if LiftingNet_noise_scale == 0:
        read_model_name = model_path[2]+model_head_name[4]+str(cutsize)+'_data_the_'+str(steps)+'th_snapshot_with_'+str(steps_per_epoch)+'_steps_per_epoch.h5'
else:
    cutsize = 256
    input_shape = (640*circle_num,3)
    channel = 3
    dataset, label = load_dataset(data_path[0], circle_num=circle_num, cutsize=cutsize)
    if circle_num<7:
        read_model_name = model_path[0] + model_head_name[0]+str(circle_num)+'_data_the_'+str(steps)+'th_snapshot_with_'+str(steps_per_epoch)+'_steps_per_epoch.h5'
    else:
        read_model_name = model_path[1] + model_head_name[1]+str(LiftingNet_noise_scale)+'_noise_'+str(circle_num)+'_data_the_'+str(steps)+'th_snapshot_with_'+str(steps_per_epoch)+'_steps_per_epoch.h5'

if circle_num==8:
    read_model_name= model_path[1] + model_head_name[3] +  str(circle_num)+'_data_the_'+str(steps)+'th_snapshot_with_'+str(steps_per_epoch)+'_steps_per_epoch.h5'

print('set adjustable parameters')

In [ ]:
#------------------------data processing---------------------------------#
input_shape = (dataset.shape[1],dataset.shape[2])
channel = dataset.shape[2]
x_number = dataset.shape[0]

x_train = dataset
y_train = label
print('x_train.shape: ', x_train.shape)
print('y_train.shape: ', y_train.shape)



CWRU_data_trans_to_us_path = '/media/silverbullet/data_and_programing_file/newpaper/dataset/CWRU_data_trans_to_our_data'
our_data_trans_to_CWRU_data_path = '/media/silverbullet/data_and_programing_file/newpaper/dataset/our_data_trans_to_CWRU_data/length_1024'


if whether_use_CWRU_data == 1:
    x_test_data_name = our_data_trans_to_CWRU_data_path+'/dataset_trans_to_CWRU_data_' + str(cutsize) + '.npy'
    y_test_data_name = our_data_trans_to_CWRU_data_path+'/label_trans_to_CWRU_data_' + str(cutsize)+'.npy'
else:
    x_test_data_name = CWRU_data_trans_to_us_path+'/CWRU_dataset_trans_to_'+str(circle_num)+'_circles_data.npy'
    y_test_data_name = CWRU_data_trans_to_us_path+'/CWRU_label_trans_to_'+str(circle_num)+'_circles_data.npy'

x_test = np.load(x_test_data_name)
y_test = np.load(y_test_data_name)

print('x_test.shape: ',x_test.shape)
print('y_test.shape: ', y_test.shape)


In [ ]:
#------------------------expansion data parameters-----------------------#

if whether_expansion_train_data == 1:
    x_train, y_train = expansion_and_add_noise(x_train,y_train,exnumber=expansion_data_number, noise_scales=LiftingNet_noise_scale,whether_link_original_data=whether_append_expansion_train_data,is_label_sort=0)
    print('data expansion')

if whether_expansion_test_data == 1:
    x_test, y_test = expansion_and_add_noise(x_test, y_test,exnumber=expansion_data_number,noise_scales=noise_scales,whether_link_original_data=whether_append_expansion_test_data,is_label_sort=0)

x_train_number = x_train.shape[0]
print('x_train_number: ',x_train_number)
x_test_number = x_test.shape[0]
print('x_test_number: ',x_test_number)